In [5]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LinearRegression

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [13]:
# Read salary data
file_path = Path("Resources/lending_data.csv")
df_lending = pd.read_csv(file_path)

# Display sample data
df_lending.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [14]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df_lending['loan_status']

# Separate the X variable, the features
X = df_lending.drop(columns=['loan_status'])

In [15]:
# Review the y variable Series
y

0        0
1        0
2        0
3        0
4        0
        ..
77531    1
77532    1
77533    1
77534    1
77535    1
Name: loan_status, Length: 77536, dtype: int64

In [16]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [17]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [18]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y)



---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [19]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [20]:
# Make a prediction using the testing data
training_predictions = lr_model.predict(X_train)
testing_predictions = logistic_regression_model.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [25]:
# Print the balanced_accuracy score of the model
from sklearn.metrics import balanced_accuracy_score

BAS = balanced_accuracy_score(y_test, testing_predictions)
print("Accuracy Score is {:.2f}".format(BAS))

Accuracy Score is 0.96


In [26]:
# Generate a confusion matrix for the model
test_matrix = confusion_matrix(y_test, testing_predictions)

# Print the confusion matrix for the testing data
print(test_matrix)

[[18697   107]
 [   48   532]]


In [27]:
# Print the classification report for the model
testing_report = classification_report(y_test, testing_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18804
           1       0.83      0.92      0.87       580

    accuracy                           0.99     19384
   macro avg       0.91      0.96      0.93     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The model performs well, with a 96% accuracy score the 83% for the precision score on the high risk loans should be considered and probably should not be used but instead refinded to bring that number up closer to 1

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [31]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
random_oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_train_resampled, y_train_resampled = random_oversampler.fit_resample(X_train, y_train)


In [39]:
# Count the distinct values of the resampled labels data
X_train_resampled.nunique()


loan_size            177
interest_rate       4340
borrower_income      638
debt_to_income       638
num_of_accounts       17
derogatory_marks       4
total_debt           638
dtype: int64

In [40]:
y_train_resampled.nunique()

2

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [41]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model_resample = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
lr_model_resampled = logistic_regression_model_resample.fit(X_train_resampled, y_train_resampled)

# Make a prediction using the testing data
testing_predictions_resampled = logistic_regression_model_resample.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [42]:
# Print the balanced_accuracy score of the model 
BAS_resampled = balanced_accuracy_score(y_test, testing_predictions_resampled)
print("Accuracy Score is {:.2f}".format(BAS))

Accuracy Score is 0.96


In [43]:
# Generate a confusion matrix for the model
test_matrix_resampled = confusion_matrix(y_test, testing_predictions_resampled)

# Print the confusion matrix for the testing data
print(test_matrix_resampled)

[[18681   123]
 [    4   576]]


In [45]:
# Print the classification report for the model
testing_report_resampled = classification_report(y_test, testing_predictions_resampled)

# Print the testing classification report
print(testing_report_resampled)
print(testing_report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18804
           1       0.82      0.99      0.90       580

    accuracy                           0.99     19384
   macro avg       0.91      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18804
           1       0.83      0.92      0.87       580

    accuracy                           0.99     19384
   macro avg       0.91      0.96      0.93     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** the precision score is lower while the recall is higher for the resampled report. overall not much difference when also considering the accuracy score as well. This shows that oversampling does not really benefit the data scientist